<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/examples_for_ppl_who_asked/Dadu_river.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dadu river

Some preliminary analyses by SMM 01-Oct-2024

## Stuff we need to do if you are in colab (not required notable or in the lsdtopotools pytools container)

The assumption for these notebooks is that you are working in a notable container.

So these steps are turned into comments using a `#` symbol before each line.

If you want to work on google colab you will need to delete the `#` symbols

In [1]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

In [2]:
!pip install lsdviztools &> /dev/null

Now import stuff you will need

In [3]:
import lsdviztools.lsdbasemaptools as bmt
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio


## Regional drainage map

Lets get a 90m regional drainage map

In [11]:
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdbasemaptools as bmt

lower_left = [29.236603892503023, 101.16909582323389]
upper_right = [31.71349674507134, 102.99166599949044]

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "RegionalDEM"
source_name = "COP90"


My_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = lower_left,
                        upper_right_coordinates = upper_right,
                        prefix = Dataset_prefix,
                        api_key_file = your_OT_api_key_file)
My_DEM.print_parameters()
My_DEM.download_pythonic()

DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

I am reading you OT API key from the file my_OT_api_key.txt
Your api key starts with: 9515
I am taking your coordinates from the lower left list
I am taking your coordinates from the upper right list
I am reading you OT API key from the file my_OT_api_key.txt
Your source is a 90m DEM.
The source is: COP90
The west longitude is: 101.16909582323389
The east longitude is: 102.99166599949044
The south latitude is: 31.71349674507134
The north latitude is: 29.236603892503023
The path is: ./
The prefix is: RegionalDEM
The resolution is: 90
I am going to download a file from opentopography (I've removed the API key):
https://portal.opentopography.org/API/globaldem?demtype=COP90&south=29.236603892503023&north=31.71349674507134&west=101.16909582323389&east=102.99166599949044&outputFormat=GTiff
This might take a little while, depending on the size of the file. 
The filename will be:
./RegionalDEM_COP90.tif
The path and file without path are:
./  RegionalDEM_COP90.tif
Finished downloading
The proj

'RegionalDEM_COP90_UTM.bil'

In [5]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"write_hillshade" : "true",
                    "print_channels_to_csv" : "true",
                    "threshold_contributing_pixels" : "5000"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

The lsdtopotools command line tools available are: 
['lsdtt-basic-metrics', 'lsdtt-channel-extraction', 'lsdtt-chi-mapping', 'lsdtt-cosmo-tool', 'lsdtt-hillslope-channel-coupling', 'lsdtt-valley-metrics']
Testing has been done against lsdtopotools v0.7
The command line tool is: lsdtt-basic-metrics
The driver name is: Test_01
The read path is: ./
The write path is: ./
The read prefix is: RegionalDEM_COP90_UTM
The write prefix is: RegionalDEM_COP90_UTM
The parameter dictionary is:
{'write_hillshade': 'true', 'print_channels_to_csv': 'true', 'threshold_contributing_pixels': '5000'}
Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.


In [10]:
!lsdtt-basic-metrics Test_01.driver

|| Welcome to the LSDTopoTools basic metrics tool!     ||
|| This program has a number of options for calculating||
|| simple landscape metrics.                           ||
|| This program was developed by Simon M. Mudd         ||
||  at the University of Edinburgh                     ||
|| If you use these routines please cite:              ||
|| http://doi.org/10.5281/zenodo.4577879               ||
|| If you use the roughness routine please cite:       ||
|| https://www.doi.org/10.5194/esurf-3-483-2015        ||
|| Documentation can be found at:                      ||
|| https://lsdtopotools.github.io/LSDTT_documentation/ ||
|| This is LSDTopoTools2 version                       ||
|| 0.9
|| If the version number has a d at the end it is a    ||
||  development version.                               ||
|| You have called an LSDTopoTools program.            ||
|| Prepare to explore topographic data!                ||
|| You can find some examples of usage here:           ||
|| http

In [ ]:
### Plot points the hillshade
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "RegionalDEM_COP90_UTM"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "elevation(m)",
                                          points_fname = "RegionalDEM_COP90_UTM_CN.csv",
                                          scale_points = True, column_for_scaling = "Stream Order",
                                          scaled_data_in_log = False, max_point_size = 20, min_point_size = 2,
                                          cmap="Blues", save_fig=True, size_format="geomorphology")

## Part 1: Get data

**First step, copy your Opentopography API key into a text file called "my_OT_api_key.txt" and make sure it is in the same file system as this notebook.**


Make sure we are using the latest version of `lsdviztools`

Double check that the version is correct. For this lesson we need version 0.4.11 or above.

We also need to import some stuff

**Next step: make sure the filename of your API key matches the one below.** Filenames are case sensitive. So if your file is "my_OT_API_key.txt" this will not work in the code block below (because api needs to be lower case).

**Then update the latitude and longitude of the lower left and upper right corner of your study area. You can get a latitude and longitude by right clicking on google maps.**

### Edit code block below for your site!

* change latitude and longitude of your `lower_left` and `upper_right` corners.
* Make sure your api filename is correct.



In [ ]:
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdbasemaptools as bmt

lower_left = [29.12163383683611, 101.13511739534259]
upper_right = [31.486686128894064, 102.31250441727097]

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "DEM"
source_name = "AW3D30"


My_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = lower_left,
                        upper_right_coordinates = upper_right,
                        prefix = Dataset_prefix,
                        api_key_file = your_OT_api_key_file)
My_DEM.print_parameters()
My_DEM.download_pythonic()

DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

**Important**: you should always cite your data source. For this data, you should cite:

*Japan Aerospace Exploration Agency (2021). ALOS World 3D 30 meter DEM. V3.2, Jan 2021. Distributed by OpenTopography. https://doi.org/10.5069/G94M92HB. Accessed: 2024-10-016*

## Part 2: Get a hillshade: **you have to do this for the rest of the plotting to work**

We are going to get a hillshade since this is used in all of the plotting routines in `lsdviztools`.

*You only need to execute this code, you don't need to change anything!*

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

This prints the image. It will also save a png file.

In [ ]:
DataDirectory = "./"
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix

this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=True,
                                 size_format="geomorphology",dpi=400,
                                 out_fname_prefix = "hillshade_plot")

print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

## Part 4: Select basins

We can select basins by giving `lsdtopotools` basin outlets.

The below code block allows you to paste in some basin outlets.

So you can change some of the outlets, make sure each outlet has two numbers, separated by a comma, in square brackets.
There needs to be a comma after each of the outlet locations, apart from the last one.
All of these get enclosed in an outer pair of square brackets.

Like this:
```
[ [a,b],
  [c,d],
  [e,f]
  ]
```

**Important**: if your DEM does not contain the entire drainage basin, the basin will not be extracted. That is, you can't get a basin that is beheaded by the edge of your map. Also you should pick an outlet slightly upstream of where you want the basin, for reasons that are too tedious to explain here.

### Edit code block below for your site!

* change the latitude and longitude of your basin outlets using the format described above. The ways you can mess this up are by not having the correct placing of square brackets and having commas missing, or in the wrong place. Follow the format above!

In [ ]:
# Import pandas library
import pandas as pd

data = [ [29.266355801779564, 102.25740103680675],
         [29.450395920667845, 102.16455080279086],
         [29.682882169543852, 102.07035651413568],
         [29.928833993423773, 101.96077845636759],
         [30.075214901697425, 102.15896043829667],
         [30.0643256870771, 101.96099504691175]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("basin_outlets.csv",index=False)
df.head()

This runs the analysis.

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"print_basin_raster" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv",
                    "print_chi_data_maps" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

This makes the plots:

In [ ]:
#import lsdviztools.lsdmapwrappers as lsdmw
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix

#%%capture
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth",
                             size_format="geomorphology",dpi=400, save_fig = True,
                             out_fname_prefix = "basins_plot")

In [ ]:
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

## Part 5: Plot channel profiles

We can look at the channel profiles in your basins.

A typical channel profile will be a plot of elevation against flow distance. But this makes it a little difficult to see changes in channel steepness. Why? Because headwater channels tend to be steeper than downstream channels. In fact, the gradient of channels is thought to scale with drainage area, so to compare relative steepness, you need to control for drainage area.

A clever way to do this is to integrate drainge area as you move upstream. I won't explain why this works by if you want to know about it please read this paper:

*Perron, J.T, and Royden, L. 2013. An Integral Approach to Bedrock River Profile Analysis. Earth Surface Processes and Landforms 38 (6): 570–76. https://doi.org/10.1002/esp.3302.*

This results in a transformed channel coordinate, which we call "chi" (it is this greek letter: $\chi$).

If you plot elevation against $\chi$, you get something that looks like a channel profile. But this profile has the very convinient feature that the gradient of the profile should be proportional to something called the channel steepness index, which in many landscapes is closely correlated with the erosion rate. See:

*Kirby, E., Whipple, K., 2001. Quantifying differential rock-uplift rates via stream profile analysis. Geology 29, 415–418. https://doi.org/10.1130/0091-7613(2001)029<0415:QDRURV>2.0.CO;2*



This first step loads the channel profile data. This came out of the little bit of code you used to get the basins.

In [ ]:
import pandas as pd
channel_file_name = r_prefix+"_chi_data_map.csv"
df = pd.read_csv(channel_file_name)
df.head()

### Edit code block below for your site!

* change the basin number to look at different basins!

In [ ]:
basin = 0

Now we can plot the $\chi$-elevation plot:

In [ ]:
import matplotlib.pyplot as plt
df_single_basin = df[(df['basin_key'] == basin)]

fig = plt.figure()
ax = fig.add_subplot(1, 1,1)

plt.scatter(df_single_basin.chi,df_single_basin.elevation,c=df_single_basin.flow_distance)
plt.xlabel(r"$\chi$ (m)")
plt.ylabel("elevation (m)")
ax.text(0.1,0.9,"Basin "+str(basin),transform=ax.transAxes)

chi_profile_name = "chi_profile_basin"+str(basin)+".png"
plt.savefig(chi_profile_name, bbox_inches='tight',dpi = 400)

In [ ]:
print(chi_profile_name)
from IPython.display import display, Image
display(Image(filename=chi_profile_name, width=800))

You can also plot the traditional channel profile.

In [ ]:
import matplotlib.pyplot as plt
df_single_basin = df[(df['basin_key'] == basin)]

fig = plt.figure()
ax = fig.add_subplot(1, 1,1)

plt.scatter(df_single_basin.flow_distance,df_single_basin.elevation,c=df_single_basin.chi)
plt.xlabel("flow distance$ (m)")
plt.ylabel("elevation (m)")
ax.text(0.1,0.9,"Basin "+str(basin),transform=ax.transAxes)

channel_profile_name = "channel_profile_basin"+str(basin)+".png"
plt.savefig(chi_profile_name, bbox_inches='tight',dpi = 400)

In [ ]:
print(channel_profile_name)
from IPython.display import display, Image
display(Image(filename=chi_profile_name, width=800))

This runs the analysis.

In [ ]:
basin = 1

Now we can plot the $\chi$-elevation plot:

In [ ]:
import matplotlib.pyplot as plt
df_single_basin = df[(df['basin_key'] == basin)]

fig = plt.figure()
ax = fig.add_subplot(1, 1,1)

plt.scatter(df_single_basin.chi,df_single_basin.elevation,c=df_single_basin.flow_distance)
plt.xlabel(r"$\chi$ (m)")
plt.ylabel("elevation (m)")
ax.text(0.1,0.9,"Basin "+str(basin),transform=ax.transAxes)

chi_profile_name = "chi_profile_basin"+str(basin)+".png"
plt.savefig(chi_profile_name, bbox_inches='tight',dpi = 400)

In [ ]:
print(chi_profile_name)
from IPython.display import display, Image
display(Image(filename=chi_profile_name, width=800))

You can also plot the traditional channel profile.

In [ ]:
import matplotlib.pyplot as plt
df_single_basin = df[(df['basin_key'] == basin)]

fig = plt.figure()
ax = fig.add_subplot(1, 1,1)

plt.scatter(df_single_basin.flow_distance,df_single_basin.elevation,c=df_single_basin.chi)
plt.xlabel("flow distance$ (m)")
plt.ylabel("elevation (m)")
ax.text(0.1,0.9,"Basin "+str(basin),transform=ax.transAxes)

channel_profile_name = "channel_profile_basin"+str(basin)+".png"
plt.savefig(chi_profile_name, bbox_inches='tight',dpi = 400)

In [ ]:
print(channel_profile_name)
from IPython.display import display, Image
display(Image(filename=chi_profile_name, width=800))

This runs the analysis.

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"print_basin_raster" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv",
                    "print_chi_data_maps" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

This makes the plots:

In [ ]:
#import lsdviztools.lsdmapwrappers as lsdmw
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix

#%%capture
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth",
                             size_format="geomorphology",dpi=400, save_fig = True,
                             out_fname_prefix = "basins_plot")

In [ ]:
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

## Part 5: Plot channel profiles

We can look at the channel profiles in your basins.

A typical channel profile will be a plot of elevation against flow distance. But this makes it a little difficult to see changes in channel steepness. Why? Because headwater channels tend to be steeper than downstream channels. In fact, the gradient of channels is thought to scale with drainage area, so to compare relative steepness, you need to control for drainage area.

A clever way to do this is to integrate drainge area as you move upstream. I won't explain why this works by if you want to know about it please read this paper:

*Perron, J.T, and Royden, L. 2013. An Integral Approach to Bedrock River Profile Analysis. Earth Surface Processes and Landforms 38 (6): 570–76. https://doi.org/10.1002/esp.3302.*

This results in a transformed channel coordinate, which we call "chi" (it is this greek letter: $\chi$).

If you plot elevation against $\chi$, you get something that looks like a channel profile. But this profile has the very convinient feature that the gradient of the profile should be proportional to something called the channel steepness index, which in many landscapes is closely correlated with the erosion rate. See:

*Kirby, E., Whipple, K., 2001. Quantifying differential rock-uplift rates via stream profile analysis. Geology 29, 415–418. https://doi.org/10.1130/0091-7613(2001)029<0415:QDRURV>2.0.CO;2*



This first step loads the channel profile data. This came out of the little bit of code you used to get the basins.

In [ ]:
import pandas as pd
channel_file_name = r_prefix+"_chi_data_map.csv"
df = pd.read_csv(channel_file_name)
df.head()

### Edit code block below for your site!

* change the basin number to look at different basins!

In [ ]:
basin = 2

Now we can plot the $\chi$-elevation plot:

In [ ]:
import matplotlib.pyplot as plt
df_single_basin = df[(df['basin_key'] == basin)]

fig = plt.figure()
ax = fig.add_subplot(1, 1,1)

plt.scatter(df_single_basin.chi,df_single_basin.elevation,c=df_single_basin.flow_distance)
plt.xlabel(r"$\chi$ (m)")
plt.ylabel("elevation (m)")
ax.text(0.1,0.9,"Basin "+str(basin),transform=ax.transAxes)

chi_profile_name = "chi_profile_basin"+str(basin)+".png"
plt.savefig(chi_profile_name, bbox_inches='tight',dpi = 400)

In [ ]:
print(chi_profile_name)
from IPython.display import display, Image
display(Image(filename=chi_profile_name, width=800))

You can also plot the traditional channel profile.

In [ ]:
import matplotlib.pyplot as plt
df_single_basin = df[(df['basin_key'] == basin)]

fig = plt.figure()
ax = fig.add_subplot(1, 1,1)

plt.scatter(df_single_basin.flow_distance,df_single_basin.elevation,c=df_single_basin.chi)
plt.xlabel("flow distance$ (m)")
plt.ylabel("elevation (m)")
ax.text(0.1,0.9,"Basin "+str(basin),transform=ax.transAxes)

channel_profile_name = "channel_profile_basin"+str(basin)+".png"
plt.savefig(chi_profile_name, bbox_inches='tight',dpi = 400)

In [ ]:
print(channel_profile_name)
from IPython.display import display, Image
display(Image(filename=chi_profile_name, width=800))

## Make the channel network

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"print_channels_to_csv" : "true",
                    "threshold_contributing_pixels" : "5000"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
!ls *.csv

In [ ]:
### Plot points the hillshade
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "DEM_AW3D30_UTM"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "elevation(m)",
                                          points_fname = "DEM_AW3D30_UTM_CN.csv",
                                          scale_points = True, column_for_scaling = "Stream Order",
                                          scaled_data_in_log = False, max_point_size = 20, min_point_size = 2,
                                          cmap="Blues", save_fig=True, size_format="geomorphology")

In [ ]:
!ls *.png